# Using dbGaPexplore, R package to explore and sort phenotypics data from dbGap

## Introduction
### Load the package

In [1]:
#install.packages("devtools", repos = "https://cran.rstudio.com/")
devtools::install_github("hms-dbmi/sandboxR", force = TRUE)
library(sandboxR)

from URL https://api.github.com/repos/hms-dbmi/sandboxR/zipball/master
Installing sandboxR
'/opt/conda/lib/R/bin/R' --no-site-file --no-environ --no-save --no-restore  \
  --quiet CMD INSTALL  \
  '/tmp/RtmpHkJ1SU/devtools2cf2c94d3/hms-dbmi-sandboxR-710c0a6'  \
  --library='/opt/conda/lib/R/library' --install-tests 



### Get the list of the function for this new package

In [2]:
lsf.str("package:sandboxR")

browse.dbgap : function (phs, jupyter = FALSE)  
browse.study : function (phs, jupyter = FALSE)  
consent.groups : function (phs)  
datatables.dict : function (phs)  
dbgap.data_dict : function (xml, dest)  
dbgap.decrypt : function (file)  
is.parent : function (phs)  
list.oldmaps : function (mappath)  
look.oldmap : function (mappath, old)  
MapToTree : function (mappath)  
n.pop : function (phs, consentgroups = TRUE, gender = FALSE)  
n.tables : function (phs)  
n.variables : function (phs)  
parent.study : function (phs)  
phs.version : function (phs)  
recover.map : function (oldcsv, mappath)  
sandbox : function (phs, consent_groups, tree_dest = consent_groups[1], study_name = phs)  
search.dbgap : function (term, jupyter = FALSE)  
study.name : function (phs)  
sub.study : function (phs)  
TreeToMap : function (mappath)  
variables.dict : function (phs)  

## 1. Search for dbGap studies
### Let's try to explore the "Jackson Heart Study" cohort that exists on dbGap.
###### The dbGap search engine can be tricky, that's why we created the function "browse.dbgap", who helps you find the studies related to the term that you search on your web browser.
Note that if you run this function in a jupyterhub environment, it will return a url since jupyterhub doesn't have access to your local browser.

In [3]:
search.dbgap("Jackson", jupyter = TRUE)

[1] "https://www.ncbi.nlm.nih.gov/gap/?term=Jackson%5BStudy+Name%5D"

#### dbGap returns the list of the studies related to your term. As you see, there are 6 studies associated with the "Jackson Heart Study" (JHS). One of these study is the main one aka the "parent study", whereas the other ones are substudies. In this case, phs000286.v5.p1 is the parent study. Firslty, we can use the phs.version() function in order to be sure that this is the latest version of the study. We can abbreviate the phs name by giving just the digit, or we can use the full dbGap id.

In [4]:
phs.version("286")

[1] "phs000286.v5.p1"

##### The is.parent() function is usefull to test if a study is a parent study or a substudy

In [5]:
is.parent("000286") # JHS main cohort
is.parent("phs499") # substudy "CARe" for JHS

[1] TRUE

[1] FALSE

#### If you don't know the parent study of a substudy, try parent.study()

In [6]:
parent.study("phs000499")

[1] "phs000286.v5.p1"                  "Jackson Heart Study (JHS) Cohort"

##### On the other side, use sub.study() to get the name and IDs of the substudies from a parent one

In [7]:
sub.study("286") # note here that the substudy "TOPMed" is missing because it has not been fully integrated yet

,phs,name
1,phs000499.v3.p1,NHLBI Jackson Heart Study Candidate Gene Association Resource (CARe)
2,phs000498.v3.p1,Jackson Heart Study Allelic Spectrum Project
3,phs000402.v3.p1,NHLBI GO-ESP: Heart Cohorts Exome Sequencing Project (JHS)
4,phs001098.v1.p1,T2D-GENES Multi-Ethnic Exome Sequencing Study: Jackson Heart Study


##### If you want to get the name of a study from its dbGap id, use study.name()

In [8]:
study.name("286")

[1] "Jackson Heart Study (JHS) Cohort"

##### Finally, you can watch your study on dbGap with browse.dbgap().
##### If a website exists for this study, you can browse it using browse.study()

In [9]:
browse.dbgap("286", jupyter = TRUE)
browse.study("286", jupyter = TRUE)

[1] "https://www.ncbi.nlm.nih.gov/projects/gap/cgi-bin/study.cgi?study_id=phs000286.v5.p1"

[1] "https://www.jacksonheartstudy.org"

## 2. Explore the characteristics of your study
##### For each dbGap study, there can be multiple consent groups that will have there specificities. Use consent.groups to know the number and the name of the consent groups in the study that you are exploring. Let's keep focusing on JHS.

In [10]:
JHS <- "phs000286"
consent.groups(JHS)

,shortName,longName
0,NRUP,"Subjects did not participate in the study, did not complete a consent document and are included only for the pedigree structure and/or genotype controls, such as HapMap subjects"
1,HMB-IRB-NPU,"Health/Medical/Biomedical (IRB, NPU)"
2,DS-FDO-IRB-NPU,"Disease-Specific (Focused Disease Only, IRB, NPU)"
3,HMB-IRB,Health/Medical/Biomedical (IRB)
4,DS-FDO-IRB,"Disease-Specific (Focused Disease Only, IRB)"


##### Use n.pop() to know the number of patient included in each groups

In [11]:
n.pop(JHS)
n.pop(JHS, consentgroups = FALSE)

,consent_group,total
1,HMB-IRB-NPU,802
2,DS-FDO-IRB-NPU,180
3,HMB-IRB,2131
4,DS-FDO-IRB,489
5,TOTAL,3602


[1] 3602

##### Use n.tables() and n.variables() to get the number of datatables in your study and the total number of variables
(n.variables may takes a while as it goes into the study files to count the actual number of variables)

In [12]:
n.tables(JHS)
n.variables(JHS)

[1] 62

[1] 4147

#### datatables.dict() will return a data frame with the datatables IDs (phtxxxxxx) and description of your study
(may takes a while)

In [13]:
tablesdict <- datatables.dict(JHS)
head(tablesdict)

,pht,dt_study_name,dt_label
1,pht002539.v2,ESP_HeartGO_JHS_Subject_Phenotypes,"Subject ID, ESP cohort, target capture used in sequencing, sequence center, race, sex, affection status, family medical history of stroke, participant medical history of asthma and COPD, ankle brachial index, artery disease status, atrioventricular block, blood pressure, body weight, height and BMI, coronary artery calcium, EKG, Framingham Risk Score, intimal-medial thickness, laboratory tests including basophils, eosinophils, neutrophils, lymphocytes, lymphocytes, blood fasting insulin and glucose, level of C-reactive protein, LDL, HDL, triglycerides, uric acid, urinary creatinine, serum creatinine, menopause, MI, FEV1, FVC, stroke status, type 2 diabetes, Wolff-Parkinson-White pattern, hormone replacement therapy, and smoking status of subjects participated in the ""National Heart Lung and Blood Institute (NHLBI) GO-ESP: Heart Cohorts Component of the Exome Sequencing Project (JHS)"" sub study of the ""Jackson Heart Study (JHS) Cohort"" project."
2,pht001948.v1,CSTA,Agatston score of all coronary section among participants of the Jackson Heart Study including adult 35-84 years old African Americans.
3,pht001947.v1,CSIA,Approach to life B. Life style among participants of the Jackson Heart Study including adult 35-84 years old African Americans.
4,pht001968.v1,PPAA,Post physical activity monitoring among participants of the Jackson Heart Study including adult 35-84 years old African Americans.
5,pht001955.v1,ECHA,Echocardiographic abnormalities among participants of the Jackson Heart Study including adult 35-84 years old African Americans.
6,pht001952.v1,DPASS_DIET1,Dietary data (DPASS) among participants of the Jackson Heart Study including adult 35-84 years old African Americans.


#### variables.dict() will return a data frame with the variables IDs (phvxxxxxx), their name in the study, the datatable where they come from and their description
(may takes even more time)

In [14]:
vardict <- variables.dict(JHS)
head(vardict)

,dt_study_name,phv,var_name,var_desc
1,ESP_HeartGO_JHS_Subject_Phenotypes,phv00165323.v2,SUBJID,Subject ID
2,ESP_HeartGO_JHS_Subject_Phenotypes,phv00165322.v2,ESP_Cohort,Cohort name [JHS]
3,ESP_HeartGO_JHS_Subject_Phenotypes,phv00165324.v2,ESP_phenotype,"ESP Phenotype group (phenotype that the sample was selected to be sequenced for) [EOMI_Control (Early MI control), LDL_Low, LDL_High, BP_Low (low blood pressure); BP_High (high blood pressure); DPR (Deeply Phenotyped Reference); BMI_High]"
4,ESP_HeartGO_JHS_Subject_Phenotypes,phv00181282.v1,Sequence_center,"Indicates where the sample was sequence at [Broad, UW]"
5,ESP_HeartGO_JHS_Subject_Phenotypes,phv00181283.v1,Target,Indicates target capture used in sequencing
6,ESP_HeartGO_JHS_Subject_Phenotypes,phv00181284.v1,ESP_race_selfreport,Self report race [African American]


Now that we have explore our datasets, let's use sandboxR in order to clean our variables, and to gather them into a tree that will be easier to use for researchers. Note that for chapter 3, we will need to move and create a lot of files on your environment. It will be easier to use on your local computer than in the Jupyterhub environment.

## 3. Extract and Build your sandboxR tree
### 3.1. Export your data from dbGap
In order to get your data from dbGap, you will need to request an access and to get a decryption key. This has to be done here: https://dbgap.ncbi.nlm.nih.gov/aa/wga.cgi?login=&page=login
### 3.2. Decrypt your files
We found that the decryption system from dbGap can be tricky. We created dbgap.decrypt() in order to easily decrypt the files that you have downloaded. Note that the "files" argument can be a file or a folder containing multiple encrypted files. Also, this function works only for Mac OS at this moment.

In [ ]:
key <- "path/to/your/key.ngc"
files <- "path/to/the/files/you/want/to/decrypt.ncbi_enc"
dbgap.decrypt(file, key)

### 3.3 Create your first sandbox map
Once the dbgap files decrypted, you will have one folder per consent groups containing one file per datatable. The goal of this function is to create a folder with a "0_map.csv" file who will map all your variables, and a subfolder "study_tree" containing one .csv file per variable in your study, gathered by datatables.

In [ ]:
cg <- c("path/to/the/first/folder/containing/a/consent/group", "second/folder", ...)
destination <- "path/were/your/tree/will/be/located"
sandbox(JHS, cg, destination)

## 4. Transform your tree and build your sandboxes
### 4.1 Modify your tree
Once your first tree has been created,  you can easily modify the arrangement of your variables by creating new subdirectories, and by dragging and dropping your variable files. You can also change the name of your directories and variables. Be careful not to delete a variable file in this process.

Then, use the function TreeToMap() in order to reflect your modifications in your "0_map.csv" file

In [ ]:
path <- "Pathway/to/the/folder/where/the/map/and/the/tree/are/located"
TreeToMap(path)

### 4.2 Modify your map
Similarly, you can modify the name of your files directly in the "0_map.csv" files. Modify the 5th column "data_label" to change the name of your variables. Use then the MapToTree() function in order to reflect your modifications in your tree.

In [ ]:
MapToTree(path)

##### Be careful to apply your changes with MapToTree or TreeToMap BEFORE switching from 4.1 to 4.2 and from 4.2 to 4.1
## 4.3 Fixing and repairing issues
Each time you will use TreeToMap(), the old map will be saved with a time stamp in a hidden folder called ".oldmaps". Use the function list.oldmaps() in order to list your previous maps. Use look.oldmaps() in order to view one of these maps as a data.frame. Use recover.map() to change your tree and your "0_map.csv" according to one of your previous maps.

In [ ]:
list.oldmaps(path)
look.oldmap(path, "olmap YYYY-MM-DD HHMM10 AM")
recover.map(path, "olmap YYYY-MM-DD HHMM AM")